In [10]:
import pandas as pd
import numpy as np

In [48]:
def disamenity_costs(radius_from, radius_to, scenario = 'low'):
    # calculates the disamenity costs in € per person, per annum, per MW installed wind capacity 
    # radius from is the inner radius in km and radius_to the outer radius of the assessed area.
  
    if scenario == 'low':
        cost_slope = -31;
        max_distance = 4;
    elif scenario == 'high':
        cost_slope = -64;
        max_distance = 8;
    else:
        print('Not a valid scenario name')
      
    assert (radius_to > 0) & (radius_from >= 0), 'radius must be positive'
    assert radius_to > radius_from, 'radius_to must be larger than radius_from'
    assert radius_to <= max_distance, f'radius_to exceeds the maximal distance'

    
    # costs per household, per year, and per windpark
    area_weighted_costs = cost_slope/2 *(radius_to**2 * (2*np.log(radius_to) -1) - (radius_from**2 * (2*np.log(radius_from) -1))) / (radius_to**2 - radius_from**2) -  cost_slope * np.log(max_distance);

    # Further assumptions
    GBP_EUR = 0.86;
    people_household = 2;
    turbines_park = 12.5;
    MW_turbine = 2;
    
    # disamenity costs
    return(area_weighted_costs / GBP_EUR / people_household / turbines_park / MW_turbine)

In [54]:
disamenity_costs(0.2,1)

56.3962678911559

In [53]:
disamenity_costs(3,4)

4.033814540850457